#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Albert_Einstein")
loader

In [4]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein', 'title': 'Albert Einstein - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nAlbert Einstein - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\

In [5]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1200,chunk_overlap=300)
documents=text_splitter.split_documents(docs)

In [6]:
documents

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein', 'title': 'Albert Einstein - Wikipedia', 'language': 'en'}, page_content='Albert Einstein - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1

In [7]:
## Must run `ollama pull llama3` before running this
from langchain_ollama import OllamaEmbeddings
embeddings=(
    OllamaEmbeddings(model="llama3")  ## 8B parameters, unless we use 'llama3:70b' for 70B parameters
    # OllamaEmbeddings(model="gemma:2b")  ## Use this for a faster model with 2B parameters

)


In [8]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [9]:
vectorstoredb

In [10]:
## Query From a vector db
query="Einstein research paper"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'1.5\n1908–1933: Academic career in Europe\n\n\n\n\n\n\n\n\n1.6\n1919: Putting general relativity to the test\n\n\n\n\n\n\n\n\n1.7\n1921–1923: Coming to terms with fame\n\n\n\n\n\n\n\n\n1.8\n1922–1932: Serving the League of Nations\n\n\n\n\n\n\n\n\n1.9\n1925: Touring South America\n\n\n\n\n\n\n\n\n1.10\n1930–1931: Touring the US\n\n\n\n\n\n\n\n\n1.11\n1933: Emigration to the US\n\n\n\n\n\n\n1.11.1\nRefugee status\n\n\n\n\n\n\n\n\n1.11.2\nResident scholar at the Institute for Advanced Study\n\n\n\n\n\n\n\n\n1.11.3\nWorld War II and the Manhattan Project\n\n\n\n\n\n\n\n\n1.11.4\nUS citizenship\n\n\n\n\n\n\n\n\n\n\n1.12\nPersonal views\n\n\n\n\n\n\n1.12.1\nPolitical views\n\n\n\n\n\n\n\n\n1.12.2\nRelationship with Zionism\n\n\n\n\n\n\n\n\n1.12.3\nReligious and philosophical views\n\n\n\n\n\n\n\n\n1.12.4\nLove of music\n\n\n\n\n\n\n\n\n\n\n1.13\nDeath\n\n\n\n\n\n\n\n\n\n\n2\nScientific career\n\n\n\n\nToggle Scientific career subsection\n\n\n\n\n\n2.1\nStatistical mechanics\n\n\n\n\n\n\n2.

In [12]:
from langchain_ollama import OllamaLLM
# llm=Ollama(model="gemma:2b")

llm=OllamaLLM(model="llama3")


In [13]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| OllamaLLM(model='llama3')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [14]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Einstein settled in America",
    "context":[Document(page_content="Einstein became an American citizen in 1940. Not long after settling into his career at the Institute for Advanced Study in Princeton, New Jersey, he expressed his appreciation of the meritocracy in American culture compared to Europe.")]
})

'Based on the provided context, it can be inferred that Einstein:\n\n* Became an American citizen in 1940\n* Settled into a career at the Institute for Advanced Study in Princeton, New Jersey (no specific date mentioned)\n* Expressed appreciation for the meritocracy in American culture compared to Europe'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [15]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [16]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [17]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fbbb00310a0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | Olla

In [18]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"Did Einstein like math ?"})
response['answer']

'Based on the provided context, what allowed Einstein to achieve so much was primarily a moral quality, specifically his deep care that the laws of physics have to explain everything in nature coherently and consistently.'

In [19]:

response

{'input': 'Did Einstein like math ?',
 'context': [Document(id='411c72b0-def0-465a-8e86-e07714747792', metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein', 'title': 'Albert Einstein - Wikipedia', 'language': 'en'}, page_content="Per Lee Smolin, I believe what allowed Einstein to achieve so much was primarily a moral quality. He simply cared far more than most of his colleagues that the laws of physics have to explain everything in nature coherently and consistently.[187] Einstein expounded his spiritual outlook in a wide array of writings and interviews.[188] He said he had sympathy for the impersonal pantheistic God of Baruch Spinoza's philosophy.[189] He did not believe in a personal god who concerns himself with fates and actions of human beings, a view which he described as naïve.[190] He clarified, however, that I am not an atheist,[191] preferring to call himself an agnostic,[192][193] or a deeply religious nonbeliever.[190] He wrote that A spirit is manifest in t

In [20]:
response['context']

[Document(id='411c72b0-def0-465a-8e86-e07714747792', metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein', 'title': 'Albert Einstein - Wikipedia', 'language': 'en'}, page_content="Per Lee Smolin, I believe what allowed Einstein to achieve so much was primarily a moral quality. He simply cared far more than most of his colleagues that the laws of physics have to explain everything in nature coherently and consistently.[187] Einstein expounded his spiritual outlook in a wide array of writings and interviews.[188] He said he had sympathy for the impersonal pantheistic God of Baruch Spinoza's philosophy.[189] He did not believe in a personal god who concerns himself with fates and actions of human beings, a view which he described as naïve.[190] He clarified, however, that I am not an atheist,[191] preferring to call himself an agnostic,[192][193] or a deeply religious nonbeliever.[190] He wrote that A spirit is manifest in the laws of the universe—a spirit vastly superior t

In [23]:
## Get the response form the LLM
test=retrieval_chain.invoke({"input":"Does Einstein like music?"})
test['answer']

"Based on the provided context, I answer:\n\nWhat was the idea that was not on Einstein's mind at any time?\n\nAnswer: The idea of becoming a professional musician himself."